In [ ]:
!pip install tensorflow==2.15.0 gdown

In [ ]:
# download the images folder 40 images
import gdown
id = "1Ki0ZV4LXLPDLsQ_knX-9c3gC2xCqbEMA"
gdown.download(id=id, output="data.zip")

In [ ]:
# Full data (image and masks)

# import gdown

# # Google Drive file ID (update this with the new ID of the zip file)
# file_id = "1OMkWH3TXdvFKhfUqVfC1sWfTArjJn3pO"

# # Download the zip file
# gdown.download(id=file_id, output="full_data.zip", quiet=False)

# # Optionally, you can unzip the downloaded file
# import zipfile
# with zipfile.ZipFile("full_data.zip", 'r') as zip_ref:
#     zip_ref.extractall("full_data")


In [ ]:
# Dowbnload the models folder
url = "https://drive.google.com/drive/folders/13zsFUu8JAZzx9tht0beD0KTNai7uZzpM?usp=drive_link"
gdown.download_folder(url)

In [ ]:
# Unzip the data files
import zipfile 

with zipfile.ZipFile("data.zip") as ref:
    ref.extractall()
    ref.close()
    

In [ ]:
# import os
# import glob

# # Define the directory path
# directory_path = '/kaggle/working/images'

# # Use glob to find all image files with the specified extensions
# image_files = glob.glob(os.path.join(directory_path, '*.jpg')) + \
#               glob.glob(os.path.join(directory_path, '*.jpeg')) + \
#               glob.glob(os.path.join(directory_path, '*.png')) + \
#               glob.glob(os.path.join(directory_path, '*.gif')) + \
#               glob.glob(os.path.join(directory_path, '*.bmp'))

# # Count the number of image files
# total_images = len(image_files)

# print(f"Total number of images: {total_images}")


In [ ]:
# Setting up the parent directory as the working directory


import tensorflow as  tf
from keras.models import load_model
import os
from PIL import Image

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
# physical_devices = tf.config.list_physical_devices('GPU')

# tf.config.set_visible_devices(physical_devices[1:], 'GPU')

In [ ]:
import tensorflow as tf

class UNet(tf.keras.Model):
    def __init__(self, in_channels, num_classes):
        super(UNet, self).__init__()
        self.down_convolution_1 = DownSample(in_channels, 64)
        self.down_convolution_2 = DownSample(64, 128)
        self.down_convolution_3 = DownSample(128, 256)
        self.down_convolution_4 = DownSample(256, 512)

        self.bottle_neck = DoubleConv(512, 1024)

        self.up_convolution_1 = UpSample(1024, 512)
        self.up_convolution_2 = UpSample(512, 256)
        self.up_convolution_3 = UpSample(256, 128)
        self.up_convolution_4 = UpSample(128, 64)

        self.out = tf.keras.layers.Conv2D(num_classes, kernel_size=1)

    def call(self, x):
        down_1, p1 = self.down_convolution_1(x)
        down_2, p2 = self.down_convolution_2(p1)
        down_3, p3 = self.down_convolution_3(p2)
        down_4, p4 = self.down_convolution_4(p3)

        b = self.bottle_neck(p4)

        up_1 = self.up_convolution_1(b, down_4)
        up_2 = self.up_convolution_2(up_1, down_3)
        up_3 = self.up_convolution_3(up_2, down_2)
        up_4 = self.up_convolution_4(up_3, down_1)

        out = self.out(up_4)
        return out


class DoubleConv(tf.keras.Model):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv_op = tf.keras.Sequential([
            tf.keras.layers.Conv2D(out_channels, kernel_size=3, padding='same'),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(out_channels, kernel_size=3, padding='same'),
            tf.keras.layers.ReLU()
        ])

    def call(self, x):
        return self.conv_op(x)


class DownSample(tf.keras.Model):
    def __init__(self, in_channels, out_channels):
        super(DownSample, self).__init__()
        self.conv = DoubleConv(in_channels, out_channels)
        self.pool = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))

    def call(self, x):
        down = self.conv(x)
        p = self.pool(down)

        return down, p


class UpSample(tf.keras.Model):
    def __init__(self, in_channels, out_channels):
        super(UpSample, self).__init__()
        self.up = tf.keras.layers.Conv2DTranspose(in_channels // 2, kernel_size=2, strides=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def call(self, x1, x2):
        x1 = self.up(x1)
        x = tf.concat([x1, x2], axis=-1)
        return self.conv(x)


In [ ]:
# For new model
model = load_model("/kaggle/working/models/unet_model_512.keras", custom_objects={'UNet' : UNet})

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import expand_dims
from tensorflow.image import resize

# Path to the folder containing images
folder_path = "/kaggle/working/images"

# Function to load and preprocess images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if img is not None:
            # Resize to 256x256
            img = resize(img, (512, 512))
            # Normalize
            img = img / 255.0
            images.append(img)
    return images

# Load images from the folder
images = load_images_from_folder(folder_path)

# Iterate through images
for image in images:
    pred_mask = model.predict(expand_dims(image, axis=0))[0]
    post_process = (pred_mask[:,:,0] > 0.5).astype('int')

    # Create a new figure for each image
    plt.figure(figsize=(15, 5))

    # Plot original image
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')

    # Plot predicted mask
    plt.subplot(1, 3, 2)
    plt.imshow(pred_mask.squeeze())
    plt.title("Predicted Mask")
    plt.axis('off')

    # Plot post-processed mask
    plt.subplot(1, 3, 3)
    plt.imshow(post_process)
    plt.title("Post-Processed Mask")
    plt.axis('off')

    # Show the figure
    plt.tight_layout()
    plt.show()


In [ ]:
# Create the mask_output folder if it doesn't exist
mask_output_folder = "/kaggle/working/mask_output"
os.makedirs(mask_output_folder, exist_ok=True)

# Iterate through images
for filename, image in zip(os.listdir(folder_path), images):
    # Predict mask
    pred_mask = model.predict(expand_dims(image, axis=0))[0]
    post_process = (pred_mask[:, :, 0] > 0.5).astype('int')

    # Save post-processed mask
    mask_filename = os.path.splitext(filename)[0] + "_mask.png"  # Append "_mask" to original filename
    mask_filepath = os.path.join(mask_output_folder, mask_filename)
    cv2.imwrite(mask_filepath, post_process * 255)  # Save the mask as a binary image

#     # Create a new figure for each image
#     plt.figure(figsize=(15, 5))

#     # Plot original image
#     plt.subplot(1, 3, 1)
#     plt.imshow(image)
#     plt.title("Original Image")
#     plt.axis('off')

#     # Plot predicted mask
#     plt.subplot(1, 3, 2)
#     plt.imshow(pred_mask.squeeze())
#     plt.title("Predicted Mask")
#     plt.axis('off')

#     # Plot post-processed mask
#     plt.subplot(1, 3, 3)
#     plt.imshow(post_process)
#     plt.title("Post-Processed Mask")
#     plt.axis('off')

    # Save the figure with the same filename as the original image
#     output_filename = os.path.splitext(filename)[0] + "_output.png"  # Append "_output" to original filename
#     output_filepath = os.path.join(mask_output_folder, output_filename)
#     plt.savefig(output_filepath)

#     # Show the figure
#     plt.tight_layout()
#     plt.show()


In [ ]:
import cv2

mask_output_folder = "/kaggle/working/mask_output_1000"
os.makedirs(mask_output_folder, exist_ok=True)

# Resize the images to 1000x1000
resized_images = [cv2.resize(img, (1000, 1000)) for img in images]

# Iterate through images
for filename, image in zip(os.listdir(folder_path), resized_images):
    # Predict mask
    pred_mask = model.predict(expand_dims(image, axis=0))[0]
    post_process = (pred_mask[:, :, 0] > 0.5).astype('int')

#     # Save post-processed mask
#     mask_filename = os.path.splitext(filename)[0] + "_mask.png"  # Append "_mask" to original filename
#     mask_filepath = os.path.join(mask_output_folder, mask_filename)
#     cv2.imwrite(mask_filepath, post_process * 255)  # Save the mask as a binary image

    # Save resized image
    output_filename = os.path.splitext(filename)[0] + "_output.png"  # Append "_output" to original filename
    output_filepath = os.path.join(mask_output_folder, output_filename)
    cv2.imwrite(output_filepath, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  # Save the resized image


In [ ]:
import cv2

mask_output_folder = "/kaggle/working/mask_output_1000"
os.makedirs(mask_output_folder, exist_ok=True)

# Ensure images are loaded correctly
if not all(isinstance(img, np.ndarray) for img in images):
    raise ValueError("Images are not loaded as NumPy arrays.")

# Ensure images are of correct shape
if not all(img.shape[0] > 0 and img.shape[1] > 0 for img in images):
    raise ValueError("Some images have invalid shapes.")

# Ensure images are of correct data type
if not all(img.dtype == np.uint8 for img in images):
    raise ValueError("Some images have invalid data type.")

# Resize the images to 1000x1000
resized_images = [cv2.resize(img, (1000, 1000)) for img in images]

# Iterate through images
for filename, image in zip(os.listdir(folder_path), resized_images):
    # Predict mask
    pred_mask = model.predict(expand_dims(image, axis=0))[0]
    post_process = (pred_mask[:, :, 0] > 0.5).astype('int')

    # Save post-processed mask
    mask_filename = os.path.splitext(filename)[0] + "_mask.png"  # Append "_mask" to original filename
    mask_filepath = os.path.join(mask_output_folder, mask_filename)
    cv2.imwrite(mask_filepath, post_process * 255)  # Save the mask as a binary image

    # Save resized image
    output_filename = os.path.splitext(filename)[0] + "_output.png"  # Append "_output" to original filename
    output_filepath = os.path.join(mask_output_folder, output_filename)
    cv2.imwrite(output_filepath, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  # Save the resized image


In [ ]:
# only process 3 images

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import expand_dims
from tensorflow.image import resize

# Path to the folder containing images
folder_path = "/kaggle/working/images"

# Function to load and preprocess images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if img is not None:
            # Resize to 256x256
            img = resize(img, (512, 512))
            # Normalize
            img = img / 255.0
            images.append(img)
    return images

# Load images from the folder
images = load_images_from_folder(folder_path)

# Only process the first 3 images
images_to_process = images[:3]

# Iterate through selected images
for i, image in enumerate(images_to_process):
    print(f"Processing image {i+1} of {len(images_to_process)}")
    pred_mask = model.predict(expand_dims(image, axis=0))[0]
    post_process = (pred_mask[:,:,0] > 0.5).astype('int')

    # Create a new figure for each image
    plt.figure(figsize=(15, 5))

    # Plot original image
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')

    # Plot predicted mask
    plt.subplot(1, 3, 2)
    plt.imshow(pred_mask.squeeze())
    plt.title("Predicted Mask")
    plt.axis('off')

    # Plot post-processed mask
    plt.subplot(1, 3, 3)
    plt.imshow(post_process)
    plt.title("Post-Processed Mask")
    plt.axis('off')

    # Show the figure
    plt.tight_layout()
    plt.show()


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import expand_dims
from tensorflow.image import resize

# Path to the folder containing images
folder_path = "/kaggle/working/images"

# Function to load and preprocess images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if img is not None:
            # Resize to 256x256
            img = resize(img, (512, 512))
            # Normalize
            img = img / 255.0
            images.append(img)
    return images

# Load images from the folder
images = load_images_from_folder(folder_path)

# Iterate through images
for image in images:
    pred_mask = model.predict(expand_dims(image, axis=0))[0]
    post_process = (pred_mask[:,:,0] > 0.5).astype('int')

    # Create a new figure for each image
    plt.figure(figsize=(15, 5))

    # Plot original image
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')

    # Plot predicted mask
    plt.subplot(1, 3, 2)
    plt.imshow(pred_mask.squeeze())
    plt.title("Predicted Mask")
    plt.axis('off')

    # Plot post-processed mask
    plt.subplot(1, 3, 3)
    plt.imshow(post_process)
    plt.title("Post-Processed Mask")
    plt.axis('off')

    # Show the figure
    plt.tight_layout()
    plt.show()


In [ ]:
#Code to process large images, splits into 1000 * 1000 then resize to 256

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import expand_dims
from tensorflow.image import resize
import tensorflow as tf

# Path to the large image
large_image_path = "/kaggle/input/chicago/41.612898753320216 -87.30889155863774_Chicago_pilot_1.tif"

# Path to save predicted masks
output_folder = "/kaggle/working/"

# Function to predict masks for large image patches
def predict_masks(image, model):
    # Define patch size
    patch_size = (1000, 1000)
    
    # Calculate number of patches in each dimension
    num_patches_h = image.shape[0] // patch_size[0] + 1
    num_patches_w = image.shape[1] // patch_size[1] + 1
    
    # Initialize empty array to store predicted masks
    predicted_masks = np.zeros(image.shape[:2], dtype=np.float32)
    
    # Iterate through patches
    for i in range(num_patches_h):
        for j in range(num_patches_w):
            # Calculate patch coordinates
            start_h = i * patch_size[0]
            end_h = min(start_h + patch_size[0], image.shape[0])
            start_w = j * patch_size[1]
            end_w = min(start_w + patch_size[1], image.shape[1])
            
            # Extract patch
            patch = image[start_h:end_h, start_w:end_w]
            
            # Resize and normalize patch
            patch = resize(patch, (256, 256)) / 255.0
            
            # Predict mask for patch
            pred_mask = model.predict(expand_dims(patch, axis=0))[0]
            post_process = (pred_mask[:,:,0] > 0.5).astype('float32')
            
            # Expand dimensions to make it 3D
            post_process = tf.expand_dims(post_process, axis=-1)
            
            # Resize the post_process array to match patch size
            post_process = tf.image.resize(post_process, [end_h - start_h, end_w - start_w])
            
            # Remove dimensions of size 1 from the shape of the tensor
            post_process = tf.squeeze(post_process)
            
            # Place predicted patch into correct location in the array
            predicted_masks[start_h:end_h, start_w:end_w] = post_process.numpy()
            
    return predicted_masks


# Load the large image
large_image = cv2.imread(large_image_path)
large_image_rgb = cv2.cvtColor(large_image, cv2.COLOR_BGR2RGB)

# Predict masks for the large image
predicted_mask = predict_masks(large_image_rgb, model)

# Save predicted mask
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "patch100 sen 5 41.612898753320216 -87.30889155863774_Chicago_pilot_1_predicted_mask.png")
plt.imsave(output_path, predicted_mask, cmap='gray')

print("Prediction saved at:", output_path)


In [ ]:
plot_model(UNet, "UNet-WaterBody.png", show_shapes=True)